### This file is for generating actionable insights from the model outputs
- for the sake of creating an MVP im going to use the data directly from dad's store
- For the real product I will pull the data from the models and use those

In [32]:
import os
import json
import logging
import polars as pl

def load_configuration(config_path):
    if not os.path.exists(config_path):
        logging.error(f"Configuration file does not exist at path: {config_path}")
        return None

    try:
        with open(config_path, 'r') as config_file:
            config = json.load(config_file)
        return config
    except json.JSONDecodeError as json_err:
        logging.error(f"Error decoding JSON from the configuration file: {json_err}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error loading configuration from {config_path}: {e}")
        return None

def convert_to_float(df, column_name):
    return df.with_columns(
        pl.col(column_name).str.replace(",", "").cast(pl.Float128).alias(column_name)
    )

def convert_to_int(df, col_name):
    # Convert to integer after handling invalid or fractional values
    return df.with_columns(
        pl.col(col_name)
        .cast(pl.Float64)  # First cast to float to handle decimals
        .round()  # Round the float values to the nearest integer
        .cast(pl.Int16)  # Finally cast to Int16
        .alias(col_name)
    )

def read_in_data_task(file_path):
    config = load_configuration('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/dashboard/configuration/InitialConfig.json')
    new_column_names = config['ColumnNames']
    json_data = None  # Initialize json_data at the start to avoid UnboundLocalError
    try:
        df = pl.read_csv(file_path, new_columns=new_column_names, infer_schema_length=1000, ignore_errors=True)
        print("DataFrame after reading CSV:")
        print(df.head(50))
        #df = convert_to_float(df, 'price')
        #print("DataFrame after converting 'price' to float:")
        #print(df.head(50))
        
        df = convert_to_float(df, 'margin')
        print("DataFrame after converting 'margin' to float:")
        print(df.head(50))
        
        for col in config['SalesColumns']:
            df = convert_to_int(df, col)
            print(f"DataFrame after converting '{col}' to int:")
            print(df.head(50))
        json_data = df.write_json()  # Convert DataFrame to JSON
    except ValueError as ve:
        print(f'ValueError during processing: {ve}')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
    
    if json_data:
        return json_data
    else:
        logging.error("Failed to process data into JSON.")
        return False
    
read_in_data_task("/Users/skylerwilson/Desktop/PartsWise/Data/Raw/PartsWise_Data.csv")

ERROR:root:Failed to process data into JSON.


DataFrame after reading CSV:
shape: (50, 33)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ part_numb ┆ descripti ┆ supplier_ ┆ quantity ┆ … ┆ sales_sep ┆ sales_oct ┆ sales_nov ┆ sales_dec │
│ er        ┆ on        ┆ name      ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---       ┆ ---       ┆ ---       ┆ i64      ┆   ┆ i64       ┆ i64       ┆ i64       ┆ i64       │
│ str       ┆ str       ┆ str       ┆          ┆   ┆           ┆           ┆           ┆           │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7547772   ┆ WELLNUT,  ┆ Polaris   ┆ 32       ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0         │
│           ┆ 6 MM X    ┆ Acceptanc ┆          ┆   ┆           ┆           ┆           ┆           │
│           ┆ 1.0 MM    ┆ e         ┆          ┆   ┆           ┆           ┆           ┆           │
│ 7519000   ┆ SCR-BTNHD ┆ Polaris   ┆ 6       

False

In [5]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/output_data/parts_data.feather")
mask = df['inventory_category'] =='essential'
df[mask]['price'].mean()

64.56730659025789

In [86]:
import pandas as pd
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Output/parts_data.feather")

# "what brand has the least total obsolete parts out of my top 5 brands with the highest gross profit?"


mask = df['inventory_category'] == 'obsolete'
qdf = df[mask].groupby('supplier_name')['quantity'].sum().sort_values()

# Calculating total gross profit per supplier
gpdf = df.groupby('supplier_name')['gross_profit'].sum().sort_values(ascending=False)

# Merging the two dataframes
fdf = pd.merge(gpdf, qdf, on='supplier_name', how='left').reset_index()

# Filter for top 5 brands by gross profit
top_5_gross_profit = fdf.nlargest(5, 'gross_profit')

# Sorting the top 5 by quantity of obsolete parts
result = top_5_gross_profit.sort_values(by='quantity', ascending=True)
result

,supplier_name,gross_profit,quantity
4,kimpex,25164.16,272.0
1,parts canada/power twins,57019.86,595.0
3,triumph motorcycles,27687.13,1418.0
2,polaris acceptance,39340.29,2808.0
0,bmw group canada,89368.96,10317.0


In [11]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/output_data/parts_data.feather")
mask = df['inventory_category'] == 'obsolete'
gm = df[~mask][['part_number', 'description', 'supplier_name', 'roi']]

# Group by supplier_name and calculate the average margin_percentage
grouped_gm = gm.groupby('supplier_name')['roi'].mean().reset_index()

# Sort by margin_percentage in descending order
grouped_gm_sorted = grouped_gm.sort_values(by='roi', ascending=False)

# Display the result
print(grouped_gm_sorted)

                   supplier_name         roi
4               bumper to bumper  670.730000
8        earthtrack supply group  419.920000
7                         ducati  318.193846
17                        lordco  198.443333
19                          moto  176.430000
18                       mavrick  176.190000
22      parts canada/power twins  149.251872
16                        kimpex   75.347229
20                       motovan   49.099219
11                     full bore    7.357500
0    action motorcycles / barnes    0.000000
28                       triumph   -7.776788
12  fundy textile and design ltd  -15.825000
25                     schuberth  -19.770000
23            polaris acceptance  -20.268584
27               thibault canada  -22.756410
3               bmw group canada  -23.011650
32              zero motorcycles  -25.000000
29           triumph motorcycles  -32.431729
26                  shell canada  -45.715000
2                bell sports inc  -48.235000
13        

In [25]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/output_data/parts_data.feather")


,part_number,description,supplier_name,quantity,price,margin,months_no_sale,quantity_ordered_ytd,special_orders_ytd,negative_on_hand,...,one_month_turnover,sell_through_rate,days_of_inventory_outstanding,order_to_sales_ratio,seasonal_component,demand,inventory_category,safety_stock,reorder_point,obsolescence_risk
3832,34527708523,o-ring,bmw group canada,339,4.99,2.27,1,0,0,0,...,0.0,1.769912,30,0.0,13.0,1.000000,essential,0,0,0.076996
8543,84010702b,"adjuster, closing rocker arm",ducati,8,24.99,24.98,3,0,0,0,...,0.0,12.500000,3650,0.0,0.0,0.999831,essential,0,0,0.253191
5143,501-170a,drain plug washer-crush,motovan,548,1.99,1.76,4,0,0,0,...,0.0,1.094891,120,0.0,19.0,0.999662,essential,0,0,0.281117
8544,84010712b,"adjuster, closing rocker arm",ducati,9,24.99,24.98,3,0,0,0,...,0.0,11.111111,3650,0.0,0.0,0.999493,essential,0,0,0.257749
8493,84010102a,"adjuster, opening rocker arm",ducati,5,11.99,11.98,3,0,0,0,...,0.0,20.000000,1825,0.0,1.0,0.999324,essential,0,0,0.703817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,46631238881,mirror base left,bmw group canada,1,76.99,27.27,18,0,0,0,...,0.0,0.000000,540,0.0,0.0,0.000000,obsolete,0,0,1.000000
4781,46631240627,cap,bmw group canada,3,8.99,3.93,18,0,0,0,...,0.0,0.000000,540,0.0,0.0,0.000000,obsolete,0,0,1.000000
4782,46631240653,clamping plate left,bmw group canada,1,6.99,2.98,18,0,0,0,...,0.0,0.000000,540,0.0,0.0,0.000000,obsolete,0,0,1.000000
4783,46631240654,clamping plate right,bmw group canada,1,8.99,3.26,18,0,0,0,...,0.0,0.000000,540,0.0,0.0,0.000000,obsolete,0,0,1.000000


In [24]:
obs_cost = df[mask].groupby('supplier_name')['total_cost'].sum()

gp = df.groupby('supplier_name')['gross_profit'].sum()

gm = df.groupby('supplier_name')['margin_percentage'].mean().reset_index()

fdf = pd.merge(obs_cost, gp, on='supplier_name', how='left').reset_index()

df = pd.merge(fdf, gm, on='supplier_name', how='left').reset_index()

df['ratio'] = df['gross_profit']/fdf['total_cost']
df.sort_values(by='ratio', ascending=False)

,index,supplier_name,total_cost,gross_profit,margin_percentage,ratio
41,41,shell canada,341.88,5556.51,53.385000,16.252808
15,15,east penn canada,54.69,166.30,51.355000,3.040775
35,35,parts canada/power twins,44941.30,82860.60,44.810040,1.843752
5,5,beeline moto,122.40,165.11,57.932500,1.348938
25,25,kimpex,19278.82,24711.89,42.150165,1.281815
28,28,mavrick,549.33,598.86,48.600000,1.090164
18,18,gamma sales inc.,317.85,207.98,36.573000,0.654334
39,39,schuberth,11101.11,6733.50,37.423000,0.606561
45,45,triumph,74055.35,37211.07,45.163483,0.502476
17,17,fundy textile and design ltd,650.89,291.63,44.803243,0.448048


In [89]:
og_df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Processed/parts_data.feather")
og_df.columns

Index(['part_number', 'description', 'supplier_name', 'quantity', 'price',
       'margin', 'months_no_sale', 'quantity_ordered_ytd',
       'special_orders_ytd', 'sales_last_jan', 'sales_last_feb',
       'sales_last_mar', 'sales_last_apr', 'sales_last_may', 'sales_last_jun',
       'sales_last_jul', 'sales_last_aug', 'sales_last_sep', 'sales_last_oct',
       'sales_last_nov', 'sales_last_dec', 'sales_jan', 'sales_feb',
       'sales_mar', 'sales_apr', 'sales_may', 'sales_jun', 'sales_jul',
       'sales_aug', 'sales_sep', 'sales_oct', 'sales_nov', 'sales_dec',
       'negative_on_hand', 'rolling_12_month_sales', 'rolling_3_month_sales',
       'cost_per_unit', 'total_cost', 'sales_revenue', 'cogs',
       'margin_percentage', 'gross_profit', 'roi', 'annual_days_supply',
       'three_month_days_supply', 'one_month_days_supply', 'annual_turnover',
       'three_month_turnover', 'one_month_turnover', 'sales_to_stock_ratio',
       'order_to_sales_ratio', 'seasonal_component', 'demand'